In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
plt.rcParams.update({'font.size': 22})

### Evaluation criterions

In this notebook the following metrics are calculated in `Cave` environments for `GBplanner` and `Naex + Trajectory optimization` with the same path follower settings: 

$V_{des} = 1.0 m/s, \Omega_{max} = 1.5 rad/s$

In [ ]:
common_metrics = [
    'Exploration face loss',
#     'Exploration edge loss',
#     'Exploration chamfer loss',
    'Exploration completeness',
#     'Map face loss',
#     'Map edge loss',
#     'Map Chamfer loss',
    'Artifacts exploration completeness',
    'Detections score',
#     'mAP',
#     'Total expected reward',
#     'Total artifacts reward',
    'Travelled distance',
    'Total actual reward',
#     'Localization error: pos',
#     'Localization error: ang'
#     'Artifacts localization error',
]

# worlds = ["finals_qual"]
worlds = ["simple_cave_01", "simple_cave_02", "simple_cave_03"]
# worlds = ["finals_qual", "simple_cave_01", "simple_cave_02", "simple_cave_03"]

#MIN_EXP_COMPL = 0.05
MIN_TRAVELLED_DIST = 100

paths = ['../data/metrics/gbplanner/gbplanner_vs_naex_trajopt/']

# all metrics files
xls_files = [os.path.join(path, file) for path in paths for file in os.listdir(path) if 'xls' in file]
    
# filter by worlds
xls_files1 = []
for file in xls_files:
    for world in worlds:
        if world in file:
            xls_files1.append(file)
xls_files = xls_files1
    
# filter by computers
computers = ['ritz', 'halmos', 'goedel']
# computers = ['halmos', 'ritz']
# computers = ['ritz']

xls_files1 = []
for file in xls_files:
    for comp in computers:
        if comp in file:
            xls_files1.append(file)
xls_files = xls_files1

In [ ]:
# names = [
#     'Time stamp',
#     'Exploration face loss',
#     'Exploration edge loss',
#     'Exploration chamfer loss',
#     'Exploration completeness',
#     'Map face loss',
#     'Map edge loss',
#     'Map chamfer loss',
#     'Artifacts exploration completeness',
#     'Detections score',
#     'mAP',
#     'Total expected reward',
#     'Total artifacts reward',
#     'Travelled distance',
#     'Total actual reward',
#     'Localization error: pos',
#     'Localization error: ang'
#     'Artifacts localization error',
# ]

# from xlrd import open_workbook
# from xlutils.copy import copy

# for filepath in xls_files:
#     rb = open_workbook(filepath)
#     wb = copy(rb)
#     s = wb.get_sheet(0)
#     for i, name in enumerate(names):
#         s.write(0, i, name)
#     wb.save(filepath)

### All experiments

In [ ]:
for world in worlds:
    for i, metric in enumerate(common_metrics):
        plt.figure(figsize=(10, 7))
        plt.title(f"World: {world}, metric: {metric}")
        xls_files_world = [file for file in xls_files if world in file]
        for file in xls_files_world:
            data = pd.read_excel(file)
            data = data.fillna(method='ffill')
            
            if metric in data and \
               np.max(data['Localization error: pos'][0:5]) < 5.0 and \
               np.max(data['Travelled distance']) > MIN_TRAVELLED_DIST and \
               np.max(data[metric]) > 0:
                ls = 'dashed' if '_gbplanner_' in file else 'solid'
                t = data['Time stamp'] - data['Time stamp'][0]
                plt.plot(t, data[metric], linestyle=ls, label=file)
                                
        plt.grid()
        plt.xlabel('time, [sec]')
        # plt.legend()
        plt.show()
        plt.close()

In [ ]:
# for key in data:
#     print(key)

In [ ]:
data.head()

### Average results

In [ ]:
xls_files_useful = []

for metric in common_metrics:
    ts, values = [], []
    ts_gbplanner, values_gbplanner = [], []
    for file in xls_files:
        data = pd.read_excel(file)
        data = data.fillna(method='ffill')

        if metric in data and \
           np.max(data['Localization error: pos'][0:5]) < 5.0 and \
           np.max(data['Travelled distance']) > MIN_TRAVELLED_DIST:
            
            t = np.array(data["Time stamp"] - data["Time stamp"][0])
            value = np.array(data[metric])

            if '_gbplanner_' in file:
                ts_gbplanner.append(t)
                values_gbplanner.append(value)
            else:
                ts.append(t)
                values.append(value)

            xls_files_useful.append(file)

    if len(xls_files_useful) == 0:
        print('No data')
        break

    t_mean = np.linspace(0, 3600, 37)

    values_interp = [np.interp(t_mean, t, v) for t, v in zip(ts, values)]
    mean = np.mean(values_interp, axis=0) 
    std = np.std(values_interp, axis=0)
    
    values_interp = [np.interp(t_mean, t, v) for t, v in zip(ts_gbplanner, values_gbplanner)]
    mean_gbplanner = np.mean(values_interp, axis=0) 
    std_gbplanner = np.std(values_interp, axis=0)

    plt.figure(figsize=(10, 7))
    plt.title(metric)

    print('Final values:')
    plt.plot(t_mean, mean, label='naex_trajopt', color='green')
    plt.fill_between(t_mean, mean-std, mean+std, color='green', alpha=0.2)
    print('Naex ' + metric + ': ' + str(mean[-1]))
    
    plt.plot(t_mean, mean_gbplanner, label='gbplanner', color='orange')
    plt.fill_between(t_mean, mean_gbplanner-std_gbplanner, mean_gbplanner+std_gbplanner, color='orange', alpha=0.2)
    print('GBplanner ' + metric + ': ' + str(mean_gbplanner[-1]))
    
    plt.grid()
    plt.xlabel('time, [sec]')
    plt.legend()
    
    # save results
    if not os.path.exists('./results/gbplanner_vs_naex_trajopt_caves/'):
        os.mkdir('./results/gbplanner_vs_naex_trajopt_caves/')
    plt.savefig('./results/gbplanner_vs_naex_trajopt_caves/' + metric + '.png')

    plt.show()
    plt.close()

In [ ]:
xls_files_useful = set(xls_files_useful)  # remove dublicates

for world_name in worlds:
    n_gbplanner = len([file for file in xls_files_useful if world_name in file and "_gbplanner_" in file])
    n_naex_trajopt = len([file for file in xls_files_useful if world_name in file and not "_gbplanner_" in file])

    print(world_name, "N gbplanner:", n_gbplanner, "N naex trajopt:", n_naex_trajopt)

In [ ]:
for cmp_name in computers:
    n_gbplanner = len([file for file in xls_files_useful if cmp_name in file and "_gbplanner_" in file])
    n_naex_trajopt = len([file for file in xls_files_useful if cmp_name in file and not "_gbplanner_" in file])

    print(cmp_name, "N gbplanner:", n_gbplanner, "N naex trajopt:", n_naex_trajopt)